In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/m5-forecasting-accuracy/calendar.csv
/kaggle/input/m5-forecasting-accuracy/sample_submission.csv
/kaggle/input/m5-forecasting-accuracy/sell_prices.csv
/kaggle/input/m5-forecasting-accuracy/sales_train_validation.csv
/kaggle/input/m5-forecasting-accuracy/sales_train_evaluation.csv


In [3]:
# === Forecasting and Submission (Simplified and Improved Baseline using 3-layer GRU) ===

import pandas as pd
import numpy as np
import pickle
from keras.models import Sequential
from keras.layers import GRU, Dense, Dropout
from keras.optimizers import Adam
from keras.callbacks import ReduceLROnPlateau, EarlyStopping
from sklearn.preprocessing import MinMaxScaler

# === Load sales data ===
sales = pd.read_csv("/kaggle/input/m5-forecasting-accuracy/sales_train_validation.csv")
sales_data = sales.iloc[:, 6:].T
sales_data.columns = sales['id'].values

# === Load and preprocess calendar features ===
calendar = pd.read_csv("/kaggle/input/m5-forecasting-accuracy/calendar.csv")
calendar_cols = ['wday', 'month', 'year', 'event_name_1', 'event_name_2', 'event_type_1', 'snap_CA', 'snap_TX', 'snap_WI']
calendar_train = calendar.iloc[:1913][calendar_cols].copy().fillna("none")
calendar_feats = pd.get_dummies(calendar_train, columns=['event_name_1', 'event_name_2', 'event_type_1'])

with open("calendar_columns.pkl", "wb") as f:
    pickle.dump(calendar_feats.columns.tolist(), f)

combined = pd.concat([sales_data.reset_index(drop=True), calendar_feats], axis=1)

# === Scale data ===
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(combined)

# === Prepare training sequences ===
input_window = 28
num_items = 30490
X_train, y_train = [], []
for i in range(input_window, scaled_data.shape[0]):
    X_train.append(scaled_data[i - input_window:i])
    y_train.append(scaled_data[i][:num_items])
X_train = np.array(X_train, dtype=np.float32)
y_train = np.array(y_train, dtype=np.float32)

# === Load calendar for inference ===
with open("calendar_columns.pkl", "rb") as f:
    calendar_columns = pickle.load(f)
calendar = pd.read_csv("/kaggle/input/m5-forecasting-accuracy/calendar.csv")
calendar_future_val = pd.get_dummies(calendar.iloc[1913:1941][calendar_cols].fillna("none"), columns=['event_name_1', 'event_name_2', 'event_type_1'])
calendar_future_val = calendar_future_val.reindex(columns=calendar_columns, fill_value=0)
calendar_next_28_val = calendar_future_val.values.astype(np.float32)

# === Define 3-layer GRU Model Architecture ===
def build_3layer_gru_model(input_timesteps, num_features, num_items, dropout_rate=0.2):
    model = Sequential()
    model.add(GRU(64, return_sequences=True, input_shape=(input_timesteps, num_features), dropout=dropout_rate, recurrent_dropout=0.1))
    model.add(GRU(128, return_sequences=True, dropout=dropout_rate, recurrent_dropout=0.1))
    model.add(GRU(128, return_sequences=False, dropout=dropout_rate, recurrent_dropout=0.1))
    model.add(Dense(num_items))
    optimizer = Adam(learning_rate=0.001)
    model.compile(optimizer=optimizer, loss='mean_squared_error')
    lr_callback = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-6, verbose=1)
    early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
    return model, [lr_callback, early_stop]

# === Train Model ===
model, callbacks = build_3layer_gru_model(28, X_train.shape[2], 30490)
model.fit(X_train, y_train, epochs=30, batch_size=32, validation_split=0.1, callbacks=callbacks, verbose=1)

# === Forecasting Function ===
def forecast_28_days(model, input_sequence, calendar_future, num_items):
    window = input_sequence.shape[0]
    input_data = input_sequence.copy()
    predictions = []
    for day in range(28):
        x_input = input_data[-window:].reshape(1, window, -1)
        y_pred = model.predict(x_input, verbose=0)
        full_pred_row = np.concatenate([y_pred[0], np.zeros(calendar_future.shape[1])]).reshape(1, -1)
        y_pred_original = scaler.inverse_transform(full_pred_row)[:, :num_items]
        next_input_row = np.concatenate([y_pred[0], calendar_future[day]])
        input_data = np.vstack([input_data, next_input_row])
        predictions.append(y_pred_original)
    return np.vstack(predictions)

# === Forecast and Save ===
def make_submission(calendar_range, ids_replace_str, output_file):
    calendar_future = pd.get_dummies(calendar.iloc[calendar_range][calendar_cols].fillna("none"), columns=['event_name_1', 'event_name_2', 'event_type_1'])
    calendar_future = calendar_future.reindex(columns=calendar_columns, fill_value=0)
    calendar_next_28 = calendar_future.values.astype(np.float32)
    last_28_days_input = scaled_data[-28:, :].astype(np.float32)
    forecast = forecast_28_days(model, last_28_days_input, calendar_next_28, num_items=30490)
    forecast = np.clip(forecast, 0, None).T
    ids = [i.replace("validation", ids_replace_str) for i in sales['id'].values]
    submission_df = pd.DataFrame(forecast, columns=[f"F{i}" for i in range(1, 29)])
    submission_df.insert(0, "id", ids)
    submission_df.to_csv(output_file, index=False)
    print(f"✅ {output_file} saved successfully!")

make_submission(slice(1913, 1941), "validation", "submission_val.csv")
make_submission(slice(1941, 1969), "evaluation", "submission_eval.csv")

# === Combine and Save Final Submission ===
submission_val = pd.read_csv("submission_val.csv")
submission_eval = pd.read_csv("submission_eval.csv")
submission_all = pd.concat([submission_val, submission_eval], axis=0).reset_index(drop=True)
sample = pd.read_csv("/kaggle/input/m5-forecasting-accuracy/sample_submission.csv")
final_submission = sample[['id']].merge(submission_all, on='id', how='left')
assert final_submission['id'].duplicated().sum() == 0
assert final_submission.isnull().sum().sum() == 0
final_submission.to_csv("final_submission.csv", index=False)
print(f"✅ Final submission saved: {len(final_submission)} rows → final_submission.csv")


2025-05-04 02:28:06.448621: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746325686.657155      31 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746325686.718786      31 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
I0000 00:00:1746325712.574219      31 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13942 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5
I0000 00:00:1746325712.574849      31 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 13942 MB memory:  -> device: 1, name: Tesla T4, pci bus id: 0000:00:05.0, compute capability:

Epoch 1/30
53/53 ━━━━━━━━━━━━━━━━━━━━ 18s 182ms/step - loss: 0.0168 - val_loss: 0.0192 - learning_rate: 0.0010
Epoch 2/30
53/53 ━━━━━━━━━━━━━━━━━━━━ 7s 126ms/step - loss: 0.0132 - val_loss: 0.0186 - learning_rate: 0.0010
Epoch 3/30
53/53 ━━━━━━━━━━━━━━━━━━━━ 7s 122ms/step - loss: 0.0126 - val_loss: 0.0185 - learning_rate: 0.0010
Epoch 4/30
53/53 ━━━━━━━━━━━━━━━━━━━━ 7s 121ms/step - loss: 0.0122 - val_loss: 0.0186 - learning_rate: 0.0010
Epoch 5/30
53/53 ━━━━━━━━━━━━━━━━━━━━ 7s 121ms/step - loss: 0.0121 - val_loss: 0.0185 - learning_rate: 0.0010
Epoch 6/30
53/53 ━━━━━━━━━━━━━━━━━━━━ 7s 120ms/step - loss: 0.0120 - val_loss: 0.0184 - learning_rate: 0.0010
Epoch 7/30
53/53 ━━━━━━━━━━━━━━━━━━━━ 7s 125ms/step - loss: 0.0118 - val_loss: 0.0185 - learning_rate: 0.0010
Epoch 8/30
53/53 ━━━━━━━━━━━━━━━━━━━━ 7s 124ms/step - loss: 0.0118 - val_loss: 0.0183 - learning_rate: 0.0010
Epoch 9/30
53/53 ━━━━━━━━━━━━━━━━━━━━ 7s 121ms/step - loss: 0.0117 - val_loss: 0.0182 - learning_rate: 0.0010
Epoch 10/